Debes crear las siguientes funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [88]:
import pandas as pd
import numpy as np
import ast
import gzip
import json
from pandas import json_normalize


In [3]:

'''Funcion para extraer datos de un archivo JSON
    Lee las lineas JSON de un archivo gzippeado, 
    y lo transforma en un dataframe de pandas.
'''
def de_json_gz_a_df(file_path):
    with gzip.open(file_path, 'rt') as f:
        data = [json.loads(line) for line in f]
    return pd.DataFrame(data)

#Aplico la funcion para el archivo especifico y lo almaceno en un dataframe
df_steam_games = de_json_gz_a_df('steam_games.json.gz') 

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:

'''Funcion para extraer datos de un archivo JSON
    Lee las lineas JSON de un archivo, 
    y lo transforma en un dataframe de pandas.
'''
def de_json_a_df(archivo):
    lista = []
    with open(archivo, encoding='utf-8') as file:
        for line in file.readlines():
            lista.append(ast.literal_eval(line))
    return pd.DataFrame(lista)

# Aplico la funcion para los archivos especificos y lo almaceno en un dataframe
df_user_reviews = de_json_a_df('user_reviews.json') 
df_user_items = de_json_a_df('users_items.json') 

In [6]:
print(df_user_items.shape)
print(df_user_items.head())

(25799, 3)
             user_id                                           user_url  \
0  76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1            js41637               http://steamcommunity.com/id/js41637   
2          evcentric             http://steamcommunity.com/id/evcentric   
3              doctr                 http://steamcommunity.com/id/doctr   
4          maplemage             http://steamcommunity.com/id/maplemage   

                                             reviews  
0  [{'funny': '', 'posted': 'Posted November 5, 2...  
1  [{'funny': '', 'posted': 'Posted June 24, 2014...  
2  [{'funny': '', 'posted': 'Posted February 3.',...  
3  [{'funny': '', 'posted': 'Posted October 14, 2...  
4  [{'funny': '3 people found this review funny',...  
(88310, 5)
             user_id  items_count           steam_id  \
0  76561197970982479          277  76561197970982479   
1            js41637          888  76561198035864385   
2          evcentric         

In [7]:

#Creamos una funcion para mostrar los tipos de columnas, cantidad de filas nulas y % de nulos
def resumen(df):
    print(f'data shape: {df.shape}')
    resumen = pd.DataFrame(df.dtypes, columns=['data type'])
    resumen['#missing'] = df.isnull().sum()
    resumen['%missing'] = df.isnull().sum()*100/len(df)
    #resumen['#unique'] = df.nunique()
    return resumen

Arrancamos con la exploracion y depuracion de Steam_games

In [ ]:
df_steam_games.head()

In [8]:
print(resumen(df_steam_games))

#Se verifica un volumen elevado de nulos

data shape: (120445, 13)
             data type  #missing   %missing
publisher       object     96362  80.004982
genres          object     91593  76.045498
app_name        object     88312  73.321433
title           object     90360  75.021794
url             object     88310  73.319773
release_date    object     90377  75.035909
tags            object     88473  73.455104
reviews_url     object     88312  73.321433
specs           object     88980  73.876043
price           object     89687  74.463033
early_access    object     88310  73.319773
id              object     88312  73.321433
developer       object     91609  76.058782


In [9]:
# Contamos las filas donde todas las columnas son nulas
filas_nulas = df_steam_games[df_steam_games.isna().all(axis=1)]

# Obtenemos el número de filas
print(len(filas_nulas))

88310


In [11]:
# Utilizamos dropna para eliminar filas donde TODAS las columnas son nulas
df_steam_games_depurado = df_steam_games.dropna(how='all')

# Verificamos el DataFrame resultante
df_steam_games_depurado

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [12]:
#Volvemos a correr el mismo analisis de nulos
print(resumen(df_steam_games_depurado))
print(df_steam_games_depurado.shape)

data shape: (32135, 13)
             data type  #missing   %missing
publisher       object      8052  25.056792
genres          object      3283  10.216275
app_name        object         2   0.006224
title           object      2050   6.379337
url             object         0   0.000000
release_date    object      2067   6.432239
tags            object       163   0.507235
reviews_url     object         2   0.006224
specs           object       670   2.084954
price           object      1377   4.285047
early_access    object         0   0.000000
id              object         2   0.006224
developer       object      3299  10.266065
(32135, 13)


In [14]:
#Analizamos la columna genres ya que sera uno de los datos mas relevantes para las funciones de la API,
# verificando los valores unicos
unique_genres = pd.unique(df_steam_games_depurado['genres'].explode())

print(unique_genres.shape)
print(type(unique_genres))
unique_genres


(23,)
<class 'numpy.ndarray'>


array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', nan, 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [15]:
#Corregimos algunos valores con ampersan y eliminamos los nulos
unique_genres[13]='Animation & Modeling'
unique_genres[19]='Design & Illustration'
unique_genres2= np.delete(unique_genres, 9, axis=0)

In [16]:
unique_genres2

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Massively Multiplayer', 'Animation & Modeling',
       'Video Production', 'Utilities', 'Web Publishing', 'Education',
       'Software Training', 'Design & Illustration', 'Audio Production',
       'Photo Editing', 'Accounting'], dtype=object)

In [17]:
#Analizo los valores unicos de la columna Tags, ya que aparentemente contiene valores comunes con Genres y puede utilizarse para rellenar nulos de Genres
unique_tags = pd.unique(df_steam_games_depurado['tags'].explode())
print(unique_tags.shape)
unique_tags

(340,)


array(['Strategy', 'Action', 'Indie', 'Casual', 'Simulation',
       'Free to Play', 'RPG', 'Card Game', 'Trading Card Game',
       'Turn-Based', 'Fantasy', 'Tactical', 'Dark Fantasy', 'Board Game',
       'PvP', '2D', 'Competitive', 'Replay Value',
       'Character Customization', 'Female Protagonist', 'Difficult',
       'Design & Illustration', 'Sports', 'Multiplayer', 'Adventure',
       'FPS', 'Shooter', 'Third-Person Shooter', 'Sniper', 'Third Person',
       'Racing', 'Early Access', 'Survival', 'Pixel Graphics', 'Cute',
       'Physics', 'Science', 'VR', 'Tutorial', 'Classic', 'Gore',
       "1990's", 'Singleplayer', 'Sci-fi', 'Aliens', 'First-Person',
       'Story Rich', 'Atmospheric', 'Silent Protagonist',
       'Great Soundtrack', 'Moddable', 'Linear', 'Retro', 'Funny',
       'Turn-Based Strategy', 'Platformer', 'Side Scroller',
       'Massively Multiplayer', 'Clicker', 'Gothic', 'Isometric',
       'Stealth', 'Mystery', 'Assassin', 'Comedy', 'Stylized', 'Co-op',
     

In [18]:

# Identifico por cada fila los valores de generos contenidos en tags

# Función para realizar la intersección teniendo en cuenta valores no listas
def intersect_with_unique_genres(tags):
    if isinstance(tags, list):
        return list(set(tags).intersection(unique_genres2))
    else:
        return []

# Creo la columna genres in tags, donde quedaran los generos que existan en los valores de tags, por cada fila
# Aplicar la función de intersección
df_steam_games_depurado['genres_in_tags'] = df_steam_games_depurado['tags'].apply(intersect_with_unique_genres)

# Muestra el DataFrame resultante
df_steam_games_depurado.head()

/var/folders/83/mc_56f2n2y7bfw5_shj4jwg80000gn/T/ipykernel_3141/3964581304.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_depurado['genres_in_tags'] = df_steam_games_depurado['tags'].apply(intersect_with_unique_genres)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,genres_in_tags
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,"[Strategy, Casual, Simulation, Action, Indie]"
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,"[Free to Play, Design & Illustration, Strategy..."
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,"[Sports, Free to Play, Casual, Simulation, Indie]"
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,"[Casual, Adventure, Action]"
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,"[Casual, Sports, Indie, Action]"


In [20]:
# Creamos una mascara para identificar las filas con nulos en 'genres'
condition = df_steam_games_depurado['genres'].isnull()

# Rellenar nulos con valores de 'genres_in_tags'
df_steam_games_depurado.loc[condition, 'genres'] = df_steam_games_depurado.loc[condition, 'genres_in_tags'].apply(lambda x: x if isinstance(x, list) else [])


               publisher                                             genres  \
88310          Kotoshiro      [Action, Casual, Indie, Simulation, Strategy]   
88311   Making Fun, Inc.               [Free to Play, Indie, RPG, Strategy]   
88312       Poolians.com  [Casual, Free to Play, Indie, Simulation, Sports]   
88313               彼岸领域                        [Action, Adventure, Casual]   
88314                NaN                    [Casual, Sports, Indie, Action]   
...                  ...                                                ...   
120440   Ghost_RUS Games              [Casual, Indie, Simulation, Strategy]   
120441            Sacada                          [Casual, Indie, Strategy]   
120442      Laush Studio                        [Indie, Racing, Simulation]   
120443          SIXNAILS                                    [Casual, Indie]   
120444               NaN  [Early Access, Simulation, Action, Indie, Adve...   

                        app_name                   

In [21]:
#Volvemos a verificar el resumen de df_steam_games_depurado
resumen(df_steam_games_depurado)

#Ya verificamos que genres esta sin nulos

data shape: (32135, 14)


,data type,#missing,%missing
publisher,object,8052,25.056792
genres,object,0,0.000000
app_name,object,2,0.006224
title,object,2050,6.379337
url,object,0,0.000000
release_date,object,2067,6.432239
tags,object,163,0.507235
reviews_url,object,2,0.006224
specs,object,670,2.084954
price,object,1377,4.285047


In [22]:
# Volvemos a verificar los valores unicos
unique_genres = pd.unique(df_steam_games_depurado['genres'].explode())

print(unique_genres.shape)
print(type(unique_genres))
unique_genres


(25,)
<class 'numpy.ndarray'>


array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Design & Illustration', 'Massively Multiplayer',
       'Education', nan, 'Animation &amp; Modeling', 'Video Production',
       'Utilities', 'Web Publishing', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Animation & Modeling', 'Accounting'], dtype=object)

In [23]:
#Realizamos algunas correcciones, nuevamente

# Reemplazar 'Animation &amp; Modeling' con 'Animation & Modeling'
df_steam_games_depurado['genres'] = df_steam_games_depurado['genres'].apply(lambda x: [genre.replace('&amp;', '&') if isinstance(genre, str) else genre for genre in x])

# Reemplazar 'Design &amp; Illustration' con 'Design & Illustration'
df_steam_games_depurado['genres'] = df_steam_games_depurado['genres'].apply(lambda x: [genre.replace('&amp;', '&') if isinstance(genre, str) else genre for genre in x])


/var/folders/83/mc_56f2n2y7bfw5_shj4jwg80000gn/T/ipykernel_3141/3757854094.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_depurado['genres'] = df_steam_games_depurado['genres'].apply(lambda x: [genre.replace('&amp;', '&') if isinstance(genre, str) else genre for genre in x])
/var/folders/83/mc_56f2n2y7bfw5_shj4jwg80000gn/T/ipykernel_3141/3757854094.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_depurado['genres'] = df_steam_games_depurado['genres'].apply(lambda x: [ge

In [25]:
# Volvemos a generar los valores unicos de genres, luego de la depuracion
unique_genres = pd.unique(df_steam_games_depurado['genres'].explode())

print(unique_genres.shape)
print(type(unique_genres))
unique_genres

(23,)
<class 'numpy.ndarray'>


array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Design & Illustration', 'Massively Multiplayer',
       'Education', nan, 'Animation & Modeling', 'Video Production',
       'Utilities', 'Web Publishing', 'Software Training',
       'Audio Production', 'Photo Editing', 'Accounting'], dtype=object)

In [26]:
unique_genres= np.delete(unique_genres, 14, axis=0)

In [28]:
print(unique_genres.shape)
unique_genres

(22,)


array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Design & Illustration', 'Massively Multiplayer',
       'Education', 'Animation & Modeling', 'Video Production',
       'Utilities', 'Web Publishing', 'Software Training',
       'Audio Production', 'Photo Editing', 'Accounting'], dtype=object)

In [29]:
#Ahora generaremos una columna dummie (True/False) por cada genero

# Crea un dataframe vacio
genres_dummies= pd.DataFrame()

# Itera sobre cada valor unico de Genres 
for genres in unique_genres:
    # Crea la columna con valor true o false por fila del dataframe df_steam_games_depurado
    genres_dummies[genres] = df_steam_games_depurado['genres'].apply(lambda x: True if x == genres else False)

genres_dummies

,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,Adventure,Racing,...,Massively Multiplayer,Education,Animation & Modeling,Video Production,Utilities,Web Publishing,Software Training,Audio Production,Photo Editing,Accounting
88310,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88311,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88312,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88313,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88314,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
120441,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
120442,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
120443,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
# Concateno las nuevas columnas dummies al DataFrame original
df_steam_games_depurado = pd.concat([df_steam_games_depurado, genres_dummies], axis=1)
df_steam_games_depurado.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,...,Massively Multiplayer,Education,Animation & Modeling,Video Production,Utilities,Web Publishing,Software Training,Audio Production,Photo Editing,Accounting
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,...,False,False,False,False,False,False,False,False,False,False
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,...,False,False,False,False,False,False,False,False,False,False
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,...,False,False,False,False,False,False,False,False,False,False
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,...,False,False,False,False,False,False,False,False,False,False
88314,NaN,"[Casual, Sports, Indie, Action]",Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,...,False,False,False,False,False,False,False,False,False,False


In [36]:

# Verificar que los valores no nulos en las columnas "title" y "app_name" sean iguales
son_iguales_no_nulos = (df_steam_games_depurado['title'].notna() & df_steam_games_depurado['app_name'].notna() & (df_steam_games_depurado['title'] == df_steam_games_depurado['app_name'])).all()

# Imprimir el resultado
if son_iguales_no_nulos:
    print("Los valores en las columnas 'title' y 'app_name' son iguales para todas las filas.")
else:
    print("Los valores en las columnas 'title' y 'app_name' no son iguales para todas las filas.")

frecuencia_iguales = (df_steam_games_depurado['title'].notna() & df_steam_games_depurado['app_name'].notna() & (df_steam_games_depurado['title'] == df_steam_games_depurado['app_name'])).sum()
frecuencia_distintos = len(df_steam_games_depurado) - frecuencia_iguales  
frecuencia_distintos = len(df_steam_games_depurado) - frecuencia_iguales  

print("Porcentaje de valores iguales:", round((frecuencia_iguales/len(df_steam_games_depurado))*100,0))

Los valores en las columnas 'title' y 'app_name' no son iguales para todas las filas.
Porcentaje de valores iguales: 92.0


In [37]:
#Considerando que el porcentaje de valores iguales es elevado y no tenemos muchos valores nulos en title 
# rellenamos los valores nulos de title con los valores de la columna app_name

df_steam_games_depurado['title'] = df_steam_games_depurado['title'].fillna(df_steam_games_depurado['app_name'])

In [38]:
resumen(df_steam_games_depurado)

data shape: (32135, 36)


,data type,#missing,%missing
publisher,object,8052,25.056792
genres,object,0,0.000000
app_name,object,2,0.006224
title,object,2,0.006224
url,object,0,0.000000
release_date,object,2067,6.432239
tags,object,163,0.507235
reviews_url,object,2,0.006224
specs,object,670,2.084954
price,object,1377,4.285047


In [39]:
#Analizamos la columna specs verificando los valores unicos
unique_specs = pd.unique(df_steam_games_depurado['specs'].explode())

print(unique_specs.shape)
print(type(unique_specs))
unique_specs

#Teniendo en cuenta que la cantidad de valores unicos es considerablemente alta...
#no lo convertiremos en columnas dummies


(41,)
<class 'numpy.ndarray'>


array(['Single-player', 'Multi-player', 'Online Multi-Player',
       'Cross-Platform Multiplayer', 'Steam Achievements',
       'Steam Trading Cards', 'In-App Purchases', 'Stats',
       'Full controller support', 'HTC Vive', 'Oculus Rift',
       'Tracked Motion Controllers', 'Room-Scale', 'Downloadable Content',
       'Steam Cloud', 'Steam Leaderboards', 'Partial Controller Support',
       'Seated', 'Standing', 'Local Co-op', 'Shared/Split Screen', nan,
       'Valve Anti-Cheat enabled', 'Local Multi-Player',
       'Steam Turn Notifications', 'MMO', 'Co-op', 'Online Co-op',
       'Captions available', 'Commentary available', 'Steam Workshop',
       'Includes level editor', 'Mods', 'Mods (require HL2)', 'Game demo',
       'Includes Source SDK', 'SteamVR Collectibles', 'Keyboard / Mouse',
       'Gamepad', 'Windows Mixed Reality', 'Mods (require HL1)'],
      dtype=object)

In [53]:
#Creamos la columna anho de lanzamiento desde el dato release_date tomando solo el anho de aquellos datos con el formato correcto
df_steam_games_depurado['release_date'] = pd.to_datetime(df_steam_games_depurado['release_date'], errors='coerce', format='%Y-%m-%d')
df_steam_games_depurado['anho_lanzamiento'] = df_steam_games_depurado['release_date'].dt.year
df_steam_games_depurado['anho_lanzamiento'] = df_steam_games_depurado['anho_lanzamiento'].astype('Int64')
resumen(df_steam_games_depurado)
df_steam_games_depurado.head()

data shape: (32135, 32)


,genres,title,release_date,tags,specs,price,early_access,id,developer,Action,...,Education,Animation & Modeling,Video Production,Utilities,Web Publishing,Software Training,Audio Production,Photo Editing,Accounting,anho_lanzamiento
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro,False,...,False,False,False,False,False,False,False,False,False,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,False,...,False,False,False,False,False,False,False,False,False,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,False,...,False,False,False,False,False,False,False,False,False,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,False,767400,彼岸领域,False,...,False,False,False,False,False,False,False,False,False,2017
88314,"[Casual, Sports, Indie, Action]",Log Challenge,NaT,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,False,...,False,False,False,False,False,False,False,False,False,<NA>


In [83]:
#Convertimos el tipo de datos de los datos numericos  

# Convierte la columna 'price' a tipo de dato float64
df_steam_games_depurado['price'] = df_steam_games_depurado['price'].replace('Free To Play', pd.NA)
df_steam_games_depurado['price'] = df_steam_games_depurado['price'].replace('Free to Play', pd.NA)
df_steam_games_depurado['price'] = pd.to_numeric(df_steam_games_depurado['price'], errors='coerce')
df_steam_games_depurado['price'] = df_steam_games_depurado['price'].astype('float64')

# Convierte la columna 'id' a tipo de dato float64
df_steam_games_depurado['id'] = pd.to_numeric(df_steam_games_depurado['id'], errors='coerce')


/var/folders/83/mc_56f2n2y7bfw5_shj4jwg80000gn/T/ipykernel_3141/3944362168.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_depurado['price'] = df_steam_games_depurado['price'].replace('Free To Play', pd.NA)
/var/folders/83/mc_56f2n2y7bfw5_shj4jwg80000gn/T/ipykernel_3141/3944362168.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_depurado['price'] = df_steam_games_depurado['price'].replace('Free to Play', pd.NA)
/var/folders/83/mc_56f2n2y7bfw5_shj4jwg80000gn/T/ipykernel_3

In [84]:
#Eliminamos las filas que no contengan el dato Id o Title (son pocas)
df_steam_games_depurado = df_steam_games_depurado.dropna(subset=['id', 'title'])

In [82]:
#Por ultimo, eliminamos las columnas no relevantes para las funciones que se crearan
columnas_a_eliminar = ['publisher', 'app_name', 'url', 'reviews_url', 'genres_in_tags', 'release_date', 'genres', 'early_access']
df_steam_games_depurado = df_steam_games_depurado.drop(columnas_a_eliminar, axis=1)
resumen(df_steam_games_depurado)


data shape: (32132, 29)


,data type,#missing,%missing
title,object,0,0.000000
tags,object,162,0.504170
specs,object,669,2.082037
price,float64,2824,8.788746
id,float64,0,0.000000
developer,object,3297,10.260799
Action,bool,0,0.000000
Casual,bool,0,0.000000
Indie,bool,0,0.000000
Simulation,bool,0,0.000000


Continuo con la depuracion de df_user_reviews

In [86]:
print(df_user_reviews.shape)
df_user_reviews.head()


(25799, 3)


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [98]:
df_user_reviews['num_elements'] = df_user_reviews['reviews'].apply(len)
print(df_user_reviews['num_elements'].value_counts())

KeyError: 'reviews'

In [94]:
#Como una de las columnas del dataframe df_user_reviews es un diccionario, guardo solo esa columna para desempaquetarlo en varias columnas
solo_reviews = json_normalize(df_user_reviews['reviews'].explode()).reset_index(drop=True)

print(solo_reviews.shape)
solo_reviews.head()

(59333, 7)


,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [95]:
duplicates = solo_reviews[solo_reviews.duplicated()]
print(duplicates)

      funny                      posted last_edited item_id  \
177     NaN                         NaN         NaN     NaN   
1114         Posted September 24, 2015.              346110   
2559    NaN                         NaN         NaN     NaN   
2894           Posted January 10, 2014.              218620   
2895           Posted January 10, 2014.              105600   
...     ...                         ...         ...     ...   
50718   NaN                         NaN         NaN     NaN   
53323   NaN                         NaN         NaN     NaN   
55425   NaN                         NaN         NaN     NaN   
57578   NaN                         NaN         NaN     NaN   
57766   NaN                         NaN         NaN     NaN   

                                              helpful recommend  \
177                                               NaN       NaN   
1114   1 of 1 people (100%) found this review helpful      True   
2559                                      

In [96]:
solo_reviews = solo_reviews.drop_duplicates()

In [97]:
# Concateno las nuevas columnas al DataFrame original, con las columnas relevantes
df_user_reviews = pd.concat([df_user_reviews[['user_id']], solo_reviews], axis=1)
print(df_user_reviews.shape)
df_user_reviews.head()

(58976, 8)


,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,evcentric,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,doctr,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,maplemage,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [10]:
df_user_items.head()
print(df_user_items.shape)


(88310, 5)


In [11]:
#Como una de las columnas del dataframe df_user_items es un diccionario, guardo solo esa columna para desempaquetarlo en varias columnas
solo_items=json_normalize(df_user_items['items'].explode())


In [12]:
# Concateno las nuevas columnas al DataFrame original
df_user_items = pd.concat([df_user_items[['user_id','steam_id']], solo_items], axis=1)
df_user_items.head()

,user_id,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,76561197970982479,10,Counter-Strike,6.0,0.0
1,js41637,76561198035864385,20,Team Fortress Classic,0.0,0.0
2,evcentric,76561198007712555,30,Day of Defeat,7.0,0.0
3,Riot-Punch,76561197963445855,40,Deathmatch Classic,0.0,0.0
4,doctr,76561198002099482,50,Half-Life: Opposing Force,0.0,0.0


In [70]:
print(resumen(df_user_reviews))

data shape: (59333, 8)
            data type  #missing   %missing
user_id        object     33534  56.518295
funny          object        28   0.047191
posted         object        28   0.047191
last_edited    object        28   0.047191
item_id        object        28   0.047191
helpful        object        28   0.047191
recommend      object        28   0.047191
review         object        28   0.047191


,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,Adventure,Racing,...,Animation & Modeling,Video Production,Utilities,Web Publishing,Education,Software Training,Design & Illustration,Audio Production,Photo Editing,Accounting
88310,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88311,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88312,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88313,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88314,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
120441,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
120442,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
120443,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
